<a href="https://colab.research.google.com/github/dmbk/Anomaly-Detection-System/blob/master/FINAL_MODEL_AAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install imageio
#!pip install tensorflow_datasets
!pip install keras-layer-normalization
from google.colab import drive
!pip install alive-progress
drive.mount('/content/drive', force_remount=True)


  Created wheel for keras-layer-normalization: filename=keras_layer_normalization-0.14.0-cp36-none-any.whl size=5268 sha256=de910ca93774ff89f5118897c3200255d5b42958abf52fd7bfb3f1c2c0974bdd
  Stored in directory: /root/.cache/pip/wheels/54/80/22/a638a7d406fd155e507aa33d703e3fa2612b9eb7bb4f4fe667
Successfully built keras-layer-normalization
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import skimage
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
import time
from os.path import join
from os import listdir
from os.path import isfile, join, isdir

#import keras
import argparse
from os.path import dirname
import cv2
from google.colab.patches import cv2_imshow
	
import statistics


import shutil

import matplotlib.pyplot as plt

from alive_progress import alive_bar


import numpy as np
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers.merge import _Merge
from keras.layers import Conv2DTranspose, ConvLSTM2D, BatchNormalization, TimeDistributed, Conv2D, Dropout, Activation, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.datasets import mnist
from keras import backend as K
from functools import partial
from keras_layer_normalization import LayerNormalization


Using TensorFlow backend.


In [0]:
class Config:
    def __init__(self, data_dir_, cwdir_name_, data_set):
        self.data_set_name = data_set
        self.data_dir = data_dir_
        self.data_set_dir = join(self.data_dir, data_set)
        self.cwdir_name = cwdir_name_
        self.cwdir = join(self.data_dir,self.cwdir_name)
        self.run_data = join(self.cwdir, "training_gan")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        self.ref_model_dir = join(self.run_data, "ref_model")
        if not os.path.exists(self.cwdir):
            os.mkdir(self.cwdir)
            os.mkdir(self.run_data)
    
        if not os.path.exists(self.run_data):
            #shutil.rmtree(self.run_data)
            os.mkdir(self.run_data)
            os.makedirs(self.image_dir, exist_ok=True)
            os.mkdir(self.ref_model_dir)

        self.curr_epoch = 0
        self.DATASET_PATH = join(self.data_set_dir,"Train/")
        self.TEST_DIR = join(self.data_set_dir,"Test/")
        self.BATCH_SIZE = 2
        self.EPOCHS = 25
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_GAN")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_GAN")

        self.retrain = 0
        self.curr_epoch = 1
        self.dim1 = 10
        self.dim2 = 256
        self.dim3 = 256
        self.dim4 = 1
        self.n_critic = 5


    def reconfig(self, new_name, batch_size = 4, epochs = 5, retrain = 0, curr_epoch = 0):
        self.cwdir_name = new_name
        self.cwdir = join(self.data_dir, self.cwdir_name)
        self.run_data = join(self.cwdir, "training_gan")
        self.image_dir = join(self.run_data,self.data_set_name,"Test/")
        self.ref_model_dir = join(self.run_data, "ref_model")

        self.BATCH_SIZE = batch_size
        self.EPOCHS = epochs
        self.GEN_MODEL_PATH = join(self.cwdir,"model_gen_Conv2DLSTM_AAE")
        self.DIS_MODEL_PATH = join(self.cwdir,"model_dis_Conv2DLSTM_AAE")

        self.curr_epoch = 0
        self.retrain = retrain
        if retrain == 0:
            print("Configuring train from scratch")
            if not os.path.exists(self.cwdir):
                os.mkdir(self.cwdir)
                os.mkdir(self.run_data)
    
            if os.path.exists(self.run_data):
                shutil.rmtree(self.run_data)
                os.mkdir(self.run_data)
                os.makedirs(self.image_dir, exist_ok=True)
        else:
            print("Configuring retrain mode\n")
            self.final_gen_path = join(self.run_data, "generator_ep-"+str(curr_epoch)+".hdf5")
            self.final_disc_path = join(self.run_data, "discriminator_ep-"+str(curr_epoch)+".hdf5")
            self.final_gan_path = join(self.run_data, "gan_ep-"+str(curr_epoch)+".hdf5")
            self.curr_epoch = curr_epoch
            print("Last epoch saved : "+str(curr_epoch))

class ModelContainer:
    def __init__(self, y0, y1):
        self.generator = y0
        self.discriminator = y1

 
conf = Config(data_dir_="/content/drive/My Drive/", cwdir_name_="Conv2DLSTM_AAE", data_set="UCSD_Anomaly_Dataset.v1p2/UCSDped1/") 


In [7]:
def get_generator():
    if conf.retrain == 1 and os.path.isfile(conf.GEN_MODEL_PATH+".json"):
        model_gen=load_model(conf.GEN_MODEL_PATH)
        return model_gen

    #init = keras.initializers.RandomNormal(stddev=0.02)
    
    print("Loading generator model")

    #input_shape=[conf.dim1, conf.dim2, conf.dim3, conf.dim4]
    #X_input = Input(input_shape)
    model = Sequential()
    model.add(TimeDistributed(Conv2D(128, (11, 11), strides=4, kernel_initializer='he_normal', padding="same", name="convT1"), batch_input_shape=[None, conf.dim1, conf.dim2, conf.dim3, conf.dim4]))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    model.add(TimeDistributed(Conv2D(64, (5, 5), strides=2, kernel_initializer='he_normal', padding="same", name="convT2")))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    # # # # #
    model.add(ConvLSTM2D(64, (3, 3), kernel_initializer='he_normal', padding="same", return_sequences=True))
    model.add(LayerNormalization())
    model.add(ConvLSTM2D(32, (3, 3), kernel_initializer='he_normal', padding="same", return_sequences=True))
    model.add(LayerNormalization())
    model.summary(line_length=150)
    return model


def get_decoder():
    model = Sequential()
    model.add(ConvLSTM2D(64, (3, 3), kernel_initializer='he_normal', padding="same", return_sequences=True, batch_input_shape=[None, conf.dim1, conf.dim2, conf.dim3, conf.dim4]))
    model.add(LayerNormalization())
    # # # # #
    model.add(TimeDistributed(Conv2DTranspose(64, (5, 5), strides=2, kernel_initializer='he_normal', padding="same", name="convT3")))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    model.add(TimeDistributed(Conv2DTranspose(128, (11, 11), strides=4, kernel_initializer='he_normal', padding="same", name="convT4")))
    model.add(LayerNormalization())
    model.add(Activation("relu"))
    model.add(TimeDistributed(Conv2D(1, (11, 11), activation="tanh", kernel_initializer='he_normal',  padding="same", name="convT5"), name="convTD5"))

    model.summary(line_length=150)
    return model

get_generator()
get_decoder()

Loading generator model
Model: "sequential_2"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                                       Output Shape                                                Param #                
time_distributed_3 (TimeDistributed)                               (None, 10, 64, 64, 128)                                     15616                  
______________________________________________________________________________________________________________________________________________________
layer_normalization_5 (LayerNormalization)                         (None, 10, 64, 64, 128)                                     256                    
______________________________________________________________________________________________________________________________________________________
activation_3 (Activation)                       